In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
 df2 = pd.read_csv('../orga-datos/datos_preprocesados/fiuba_6_avisos_detalle.csv')

In [3]:
data1=df2["descripcion"][43]
data1


'<p><span style="">Nos encontramos en la búsqueda de un Analista de Créditos Jr sector Agro, para Importante Entidad Bancaria.</span></p><p>\xa0</p><p>\xa0</p><p><span style="">En esta posición, tus responsabilidades serán:</span></p><ul><li><p><span style="">Análisis y evaluación de propuestas de créditos</span></p></li><li><p><span style="">Participación conjunta en Comité de Créditos</span></p></li><li><p><span style="">Administración de cartera</span></p></li></ul><p>\xa0</p><p>\xa0</p><p>Requisitos:</p><p>\xa0</p><p><span style="">Postulate si:</span></p><ul><li><p><span style="">Sos profesional o estudiante avanzado de las carreras de Ciencias Económicas / Licenciado en Administración Agraria / Ingeniero Agrónomo</span></p></li><li><p><span style="">Tenés actitud proactiva y muy buen manejo de relaciones interpersonales para trabajar en equipo y orientación a resultados</span></p></li><li><p><span style="">Contás con muy buen manejo del Paquete Office</span></p></li></ul><p>\xa0<

In [4]:
stopswords="a acá ahí ajena ajeno ajenos ajenas al algo algún algúna algúno algúnos algúnas allá allí ambos ante antes aquel aquella aquello aquellos aquellas aquí arriba así atrás aun aunque bajo bastante bien cabe cada casi cierto cierta ciertos ciertas como con conmigo conseguimos conseguir consigo consigue consiguen consigues contigo contra cual cuales cualquier cualquiera cualquieras cuancuando cuanto cuantas cuanta cuantos de dejar del demás demasiada demasiadas demasiado demasiados dentro desde donde dos e el él ella ellas ello ellos empleáis emplean emplear empleas empleo en encima entonces entre era eras eramos eran eres es esa ese eso esos esas esta estas estaba estado estáis estamos están estar este esto estos estas estoy etc fin fue fueron fui fuimos gueno ha hace haces hacéis hacemos hacen hacer hacia hago hasta incluso intenta intentas intentáis intentamos intentan intentar intento ir jamás junto juntos la lo las los largo más me menos mi mis mía mías mientras mío míos misma mismo mismos mismas modo mucha muchas muchísima muchísimos muchísimo muchísimas mucho muchos muy nada ni ningún ningúna ningúnas ningúno ningúnos no nos nosotras nosotros nuestra nuestro nuestros nuestras nunca o os otra otro otras otros para parecer pero poca poco pocas pocos podéis podemos poder podría podrías podríais podríamos podrían por por qué porque primero puede pueden puedo pues que qué querer quién quiénes quienesquiera quienquiera quizá quizás sabe sabes saben sabéis sabemos saber se según ser si sí siempre siendo sin sino so sobre sois solamente solo sólo somos soy sr sra sres sta su sus suyo suyos suya suyas tal tales también tampoco tan tanta tanto tantos tantas te tenéis tenemos tener tengo ti tiempo tiene tienen todo todos toda todas tomar trabaja trabajo trabajáis trabajamos trabajan trabajar trabajas tras tú tu tus tuya tuyas tuyo tuyos último ultimo un una unas uno unos usa usas usáis usamos usan usar uso usted ustedes va van vais valor vamos varias varios vaya verdadera vosotras vosotros voy vuestra vuestros vuestro vuestras y ya yo como donde requisitos requisito buscamos importante buenas muy grandes beneficios "
stopswords=stopswords.split()

In [5]:
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed =[]
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ' '.join(self.fed)

def strip_html_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

def foldear_simbolos(cadena):
    ''' 
    Elimina los acentos de una cadena en minúsculas. 
    Ej: descripción -> descripcion
    '''
    acentos = {'á':'a', 'é':'e', 'í':'i', 'ó':'o', 'ú':'u', 'ü':'u'}
    cadena = list(cadena)
    for i in range(len(cadena)):
        if not cadena[i].isalpha():
            cadena[i] = ' '
        else:
            cadena[i] = acentos.get(cadena[i], cadena[i])
    
    return ''.join(cadena)

def es_palabra_inutil(palabra):
    '''
    Devuelve True si palabra es una palabra que no aporta
    contenido.
    '''
    return palabra in set(stopswords)

def parse(descripcion):
    '''
    Normaliza una descripción de aviso del set de datos de Navent.
    Elimina tags HTML, pasa a minúsculas y la convierte a tokens.
    '''
    resultado = []
    
    for palabra in strip_html_tags(descripcion).lower().split():
        if es_palabra_inutil(palabra):
            continue
            
        resultado.extend(foldear_simbolos(palabra).split())
    
    return ' '.join(resultado)
    
  

In [6]:
words={}
def total_freq(description):
    #it create a dictionary with the following format (word : #appearances in the sum of descriptions)
    words_processed=[]
    for word in description.split():
        if word not in words_processed:
            words[word] = words.get(word, 0) + 1
            words_processed.append(word)
    return description

factor = df2["descripcion"].count()*0.01 # %1 aca va el porcentaje que se quiere tener

def remove_less_frequent_words(description):
    # we will remove all the words which appear in less than 1% of the descrptions
    result = []
    for word in description.lower().split():
        if words[word] < factor:
            continue
            
        result.append(word)
    
    return ' '.join(result)

In [7]:
df2["descripcion"]=df2["descripcion"].map(parse)

In [8]:
df2["descripcion"]=df2["descripcion"].map(total_freq)
s = sorted(words, key=words.get, reverse=True)#mayor a menor frecuencia
print(s[:50])
print("****************************************************************************************************************")
s = sorted(words, key=words.get, reverse=False)#menor a mayor frecuencia
print(s[:50])

['experiencia', 'busqueda', 'empresa', 'años', 'zona', 'tareas', 'equipo', 'trabajo', 'excluyente', 'manejo', 'hs', 'lunes', 'disponibilidad', 'conocimientos', 'laboral', 'principales', 'viernes', 'cliente', 'contratacion', 'horario', 'condiciones', 'requisitos', 'clientes', 'area', 'responsabilidades', 'empresas', 'time', 'desarrollo', 'edad', 'servicios', 'seran', 'gestion', 'capacidad', 'excelente', 'administracion', 'contar', 'lugar', 'cv', 'personal', 'ingles', 'realizar', 'full', 'orientacion', 'puesto', 'encontramos', 'encuentra', 'posicion', 'secundario', 'nivel', 'ofrece']
****************************************************************************************************************
['bcom', 'envasamientos', 'parametrizado', 'entidadesfinancieras', 'zendesck', 'lapatagonica', 'desobstruccion', 'hedging', 'xbox', 'hifu', 'ingeniara', 'caracterizandonos', 'ifttt', 'escencial', 'faevyt', 'interpreting', 'belgano', 'disuadir', 'siporte', 'volanteros', 'salio', 'chaqueta', 'mobilf

In [9]:
number_of_unique_words = len(words)
print(number_of_unique_words)

36663


In [10]:
contador=0
for word in words:
    contador += words[word]
    
average_words_by_description = contador/df2["descripcion"].count()

print(average_words_by_description)  

72.9837867763


In [21]:
#print the number of words whose frequencies are bigger than 1%
number_of_words = 0
factor = df2["descripcion"].count()*0.01 # %1
for x in words:
    if words[x] > factor:
        number_of_words +=1
print(number_of_words)    

#posible problema

1222


In [12]:
df2["descripcion"] = df2["descripcion"].map(remove_less_frequent_words)

In [13]:
data1=df2["descripcion"][43]
data1

'encontramos busqueda analista sector posicion responsabilidades seran analisis evaluacion participacion administracion cartera requisitos postulate sos profesional estudiante avanzado carreras ciencias economicas administracion ingeniero actitud proactiva buen manejo relaciones interpersonales equipo orientacion resultados buen manejo paquete office conocimientos experiencia laboral analisis beneficios condiciones contratacion lunes viernes jornada full time lugar trabajo microcentro empresa ofrece condiciones contratacion'

'encontramos busqueda analista creditos jr sector agro entidad bancaria posicion responsabilidades seran analisis evaluacion propuestas creditos participacion conjunta comite creditos administracion cartera requisitos postulate si sos profesional estudiante avanzado carreras ciencias economicas licenciado administracion agraria ingeniero agronomo tenes actitud proactiva buen manejo relaciones interpersonales equipo orientacion resultados contas buen manejo paquete office valoraremos conocimientos segmento agro experiencia laboral analisis balance riesgo credito beneficios condiciones contratacion lunes viernes jornada full time lugar trabajo microcentro empresa ofrece condiciones contratacion'


In [14]:
# sudo pip3 install datasketch -U

from datasketch import MinHash

#te da lasdescripciones que tengan semejanza mayor a 0.3 (este es un buen factor)

data1=df2["descripcion"][43]
data1=data1.split()
    
def jac(texto):
    
    data2=texto.split()
    m1, m2 = MinHash(), MinHash()
    for d in data1:
        m1.update(d.encode('utf8'))
    for d in data2:
        m2.update(d.encode('utf8'))
    if m1.jaccard(m2)> 0.3:     
        print("Estimated Jaccard for data1 and data2 is", m1.jaccard(m2))
        print(" ".join(data2))

d=df2["descripcion"].map(jac)  

Estimated Jaccard for data1 and data2 is 1.0
encontramos busqueda analista sector posicion responsabilidades seran analisis evaluacion participacion administracion cartera requisitos postulate sos profesional estudiante avanzado carreras ciencias economicas administracion ingeniero actitud proactiva buen manejo relaciones interpersonales equipo orientacion resultados buen manejo paquete office conocimientos experiencia laboral analisis beneficios condiciones contratacion lunes viernes jornada full time lugar trabajo microcentro empresa ofrece condiciones contratacion
Estimated Jaccard for data1 and data2 is 0.453125
encontramos busqueda lider cobranzas compañia financiera grupo reconocida posicion responsabilidades seran grupo tarea realizar indicadores seguimiento reportes postulate sos profesional estudiante carreras ciencias economicas afin actitud proactiva buen manejo relaciones interpersonales orientacion resultados buen manejo paquete office conocimiento manejo sistema condicion

In [19]:
#no usar
from datasketch import MinHash
#data1 = ['minhash', 'is', 'a', 'probabilistic', 'data', 'structure', 'for',
#        'estimating', 'the', 'similarity', 'between', 'datasets']
#data2 = ['minhash', 'is', 'a', 'probability', 'data', 'structure', 'for',
#        'estimating', 'the', 'similarity', 'between', 'documents']
data1=data1.split()
data2=data2.split()
m1, m2 = MinHash(), MinHash()
for d in data1:
    m1.update(d.encode('utf8'))
for d in data2:
    m2.update(d.encode('utf8'))
print("Estimated Jaccard for data1 and data2 is", m1.jaccard(m2))

s1 = set(data1)
s2 = set(data2)
actual_jaccard = float(len(s1.intersection(s2)))/float(len(s1.union(s2)))
print("Actual Jaccard for data1 and data2 is", actual_jaccard)

Estimated Jaccard for data1 and data2 is 0.1328125
Actual Jaccard for data1 and data2 is 0.10588235294117647


Hasta aca es sin tf-idf ahora se le aplica tf-idf para calcular las palabras mas importantes 
y quedarme solo con esas palabras

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from scipy.sparse.csr import csr_matrix #need this if you want to save tfidf_matrix

#tf = TfidfVectorizer(input='/home/iglesias/test', analyzer='word', ngram_range=(1,6),
#min_df = 0, stop_words = 'english', sublinear_tf=True)
tf = TfidfVectorizer() 
tfidf_matrix =  tf.fit_transform(df2["descripcion"])


feature_names = tf.get_feature_names()

"""doc =0
feature_index = tfidf_matrix[doc,:].nonzero()[1]
tfidf_scores = zip(feature_index, [tfidf_matrix[doc, x] for x in feature_index])
for w, s in [(feature_names[i], s) for (i, s) in tfidf_scores]:
  print(w, s)
"""
factor=df2["descripcion"].count()

scores=[]
dos_mil_palabras_importantes = []
for doc in range(factor):
    feature_index = tfidf_matrix[doc,:].nonzero()[1]
    tfidf_scores = zip(feature_index, [tfidf_matrix[doc, x] for x in feature_index])
    for w, s in [(feature_names[i], s) for (i, s) in tfidf_scores]:
      if not w in dos_mil_palabras_importantes:   
          scores.append(-s)
          dos_mil_palabras_importantes.append(w)  

import heapq

scores_mas_importantes=[]
heapq.heapify(scores)

cantidad_de_palabras_que_quiero = 500 #fijarse de tener esta cantidad de palabras sino rompe

for x in range(cantidad_de_palabras_que_quiero):
    scores_mas_importantes.append(heapq.heappop(scores))

dos_mil_palabras_importantes = []    
    
for doc in range(df2["descripcion"].count()):
    feature_index = tfidf_matrix[doc,:].nonzero()[1]
    tfidf_scores = zip(feature_index, [tfidf_matrix[doc, x] for x in feature_index])
    for w, s in [(feature_names[i], s) for (i, s) in tfidf_scores]:
      if -s in scores_mas_importantes:   
          dos_mil_palabras_importantes.append(w) 
            
print("ok")        

ok


In [16]:
data2=df2["descripcion"][43]
data2

'encontramos busqueda analista sector posicion responsabilidades seran analisis evaluacion participacion administracion cartera requisitos postulate sos profesional estudiante avanzado carreras ciencias economicas administracion ingeniero actitud proactiva buen manejo relaciones interpersonales equipo orientacion resultados buen manejo paquete office conocimientos experiencia laboral analisis beneficios condiciones contratacion lunes viernes jornada full time lugar trabajo microcentro empresa ofrece condiciones contratacion'

In [17]:
def mas_impor(description):
    
    resultado = []
    
    for palabra in description.lower().split():
        if palabra not in dos_mil_palabras_importantes:
            continue
            
        resultado.append(palabra)
    
    return ' '.join(resultado)

In [18]:
df2["descripcion"]=df2["descripcion"].map(mas_impor)

In [19]:
data2=df2["descripcion"][43]
data2

'encontramos busqueda analista sector posicion evaluacion participacion cartera requisitos postulate sos profesional estudiante avanzado ciencias economicas actitud proactiva buen manejo relaciones interpersonales equipo buen manejo conocimientos beneficios lunes jornada full time lugar trabajo microcentro'

In [20]:
from datasketch import MinHash

data1=df2["descripcion"][43]
data1=data1.split()
    
def jac(texto):
    
    data2=texto.split()
    m1, m2 = MinHash(), MinHash()
    for d in data1:
        m1.update(d.encode('utf8'))
    for d in data2:
        m2.update(d.encode('utf8'))
    if m1.jaccard(m2)> 0.3:     
        print("Estimated Jaccard for data1 and data2 is", m1.jaccard(m2))
        print(" ".join(data2))

d=df2["descripcion"].map(jac)  

Estimated Jaccard for data1 and data2 is 1.0
encontramos busqueda analista sector posicion evaluacion participacion cartera requisitos postulate sos profesional estudiante avanzado ciencias economicas actitud proactiva buen manejo relaciones interpersonales equipo buen manejo conocimientos beneficios lunes jornada full time lugar trabajo microcentro
Estimated Jaccard for data1 and data2 is 0.53125
encontramos busqueda lider cobranzas compañia financiera grupo reconocida posicion grupo realizar indicadores reportes postulate sos profesional estudiante ciencias economicas afin actitud proactiva buen manejo relaciones interpersonales buen manejo manejo lunes jornada full time lugar trabajo microcentro
Estimated Jaccard for data1 and data2 is 0.34375
encontramos busqueda analista gerencia empresas perfil requerido profesional estudiante avanzado ciencias economicas contador publico buen manejo relaciones interpersonales equipo actitud proactiva estudios intermedio microsoft cartera cliente